In [2]:
import nltk
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

In [4]:
df = pd.read_excel('Bank (1).xlsx')
df.head()

,Unnamed: 0,Sentences for Training,"Sense of the word ""Bank"""
0,NaN,I deposited my paycheck at the bank.,Financial Institution
1,NaN,The bank of the river was eroded by the heavy ...,River Border
2,NaN,She works at the bank as a teller.,Financial Institution
3,NaN,Let's have a picnic by the bank of the river.,River Border
4,NaN,The bank approved my loan application.,Financial Institution


In [6]:
df = df.drop(columns=['Unnamed: 0'])
df.head()

,Sentences for Training,"Sense of the word ""Bank"""
0,I deposited my paycheck at the bank.,Financial Institution
1,The bank of the river was eroded by the heavy ...,River Border
2,She works at the bank as a teller.,Financial Institution
3,Let's have a picnic by the bank of the river.,River Border
4,The bank approved my loan application.,Financial Institution


In [8]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalpha()]
    stemmer = PorterStemmer()
    tokens =[stemmer.stem(token) for token in tokens]
    return tokens

In [15]:
df.columns
df['tokens'] = df['Sentences for Training '].apply(preprocess_text)
print(df.head())

                             Sentences for Training  Sense of the word "Bank"  \
0               I deposited my paycheck at the bank.    Financial Institution   
1  The bank of the river was eroded by the heavy ...             River Border   
2                She works at the bank as a teller.     Financial Institution   
3     Let's have a picnic by the bank of the river.              River Border   
4            The bank approved my loan application.     Financial Institution   

                                              tokens  
0          [i, deposit, my, paycheck, at, the, bank]  
1  [the, bank, of, the, river, wa, erod, by, the,...  
2          [she, work, at, the, bank, as, a, teller]  
3  [let, have, a, picnic, by, the, bank, of, the,...  
4              [the, bank, approv, my, loan, applic]  


In [20]:
df.columns = ['Sentences','Sense','tokens']

In [50]:
df.head()
unique_words = []
unique_words = [word for words in df['tokens'] for word in words if word not in unique_words]
N = len(unique_words)


In [25]:
unique_sense = df['Sense'].unique()
num_class = len(unique_sense)
num_class

2

In [26]:
total_sent = len(df)
total_sent

93

In [33]:
sense_count=df['Sense'].value_counts().to_dict()

In [36]:
prior = {}
for i in unique_sense:
    prob = sense_count[i]/total_sent
    prior[i] = prob
prior

{'Financial Institution': 0.4838709677419355,
 'River Border': 0.5161290322580645}

In [37]:
s = input("Enter the Input Sentence")
s = preprocess_text(s)
s

Enter the Input Sentence i had a bad game


['i', 'had', 'a', 'bad', 'game']

In [42]:
d = {}
for i in s:
    d[i] = {'Financial Institution':0,'River Border':0}
    for idx,row in df.iterrows():
        if i in row['tokens']:
            d[i][row['Sense']] += 1
print(d)

{'i': {'Financial Institution': 15, 'River Border': 0}, 'had': {'Financial Institution': 0, 'River Border': 0}, 'a': {'Financial Institution': 12, 'River Border': 10}, 'bad': {'Financial Institution': 0, 'River Border': 0}, 'game': {'Financial Institution': 0, 'River Border': 0}}


In [52]:
probability = {}
for i in d:
    probability[i] = {'Financial Institution':0,'River Border':0}
    prob = (d[i]['Financial Institution'] + 1)/(sum(d[i].values()) + N)
    probability[i]['Financial Institution'] = prob

for i in d:
    prob = (d[i]['River Border'] + 1)/(sum(d[i].values()) + N)
    probability[i]['River Border'] = prob
probability

{'i': {'Financial Institution': 0.016260162601626018,
  'River Border': 0.0010162601626016261},
 'had': {'Financial Institution': 0.0010319917440660474,
  'River Border': 0.0010319917440660474},
 'a': {'Financial Institution': 0.013118062563067608,
  'River Border': 0.011099899091826439},
 'bad': {'Financial Institution': 0.0010319917440660474,
  'River Border': 0.0010319917440660474},
 'game': {'Financial Institution': 0.0010319917440660474,
  'River Border': 0.0010319917440660474}}

In [55]:
print(prior)

{'Financial Institution': 0.4838709677419355, 'River Border': 0.5161290322580645}


In [62]:
import math
Financial_prob = math.log(prior['Financial Institution'],2)
River_prob = math.log(prior['River Border'],2)
for i in probability:
    Financial_prob += math.log(probability[i]['Financial Institution'],2)
    River_prob += math.log(probability[i]['River Border'],2)
Financial_prob 


-43.00318031540824

In [63]:
River_prob

-47.151079010520554

In [65]:
if Financial_prob > River_prob :
    print('Sense of the Sentence is Finanical Institution')
else:
    print('Sense of the Sentence is River Border')

Sense of the Sentence is Finanical Institution
